介绍自动微分

In [1]:
import torch
x = torch.arange(4.0)
x


tensor([0., 1., 2., 3.])

一个标量函数关于向量的梯度是向量，并且与具有相同的形状

In [2]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)启用梯度追踪
x.grad  # 默认值是None

In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

调用反向传播函数来自动计算y关于x每个分量的梯度

In [4]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

<pre>
当x改变时，再次得到梯度  
    当执行 y = 2 * torch.dot(x, x) 时，PyTorch 会记录x到y的计算路径。如果后续直接修改x的值（如 x[0] = 5），  
而没有重新构建计算图，backward()可能会基于旧的计算图（包含原始 x 的值）计算梯度，导致错误  
    安全做法：  
        重新创建 x（如 x = torch.tensor(...)），或  
        通过 x.data 修改值(谨慎使用)，或  
        在 torch.no_grad() 上下文中修改。  
    始终记得清空梯度（x.grad.zero_()），避免累加。  
注意：  
    每次都要重新定义y的表达值
<pre>


In [5]:
x = torch.tensor([5., 1., 2., 3.], requires_grad=True)  # 直接用新值重建 x
y = 2 * torch.dot(x, x)
y.backward()
x.grad

tensor([20.,  4.,  8., 12.])

In [6]:
# 通过 x.data 修改值（避免影响计算图）
x.grad.zero_()
x.data[0] = 5  # 修改底层数据，不破坏计算图
y = 2 * torch.dot(x, x)  # 重新计算 y
y.backward()
x.grad

tensor([20.,  4.,  8., 12.])

In [7]:
# 在无梯度追踪的上下文中修改 x
with torch.no_grad():
    x[0] = 10
x.grad.zero_()
y = 2 * torch.dot(x, x)
y.backward()
x.grad

tensor([40.,  4.,  8., 12.])

如果需要多次计算梯度，建议将逻辑封装为函数或循环：

In [8]:
def compute_gradient(x):
    if x.grad is not None:
        x.grad.zero_()
    y = 2 * torch.dot(x, x)
    y.backward()
    return x.grad

# 初始计算
x = torch.arange(4.0, requires_grad=True)
print("第一次梯度:", compute_gradient(x))

# 修改 x 并重新计算
x.data.add_(1)
print("修改后梯度:", compute_gradient(x))

第一次梯度: tensor([ 0.,  4.,  8., 12.])
修改后梯度: tensor([ 4.,  8., 12., 16.])


当y关于x的表达式发生变化时需要先梯度清零

In [9]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

 对于高阶和高维的y和x，求导的结果可以是一个高阶张量  
 backward()调用对象需要是标量  
 好在机器学习里只是计算样本批量的梯度之和，再作平均  
 因此只需对y进行求和变成标量即可  
 

In [10]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([2., 4., 6., 8.])

y.backward()需要传入的的参数  
当y是张量（多维）时，必须提供一个与y形状相同的gradient参数，指定如何将 y 的每个分量加权求和得到标量，才能反向传播。  
y是标量（单个值）时，可以直接调用 y.backward()（等价于 y.backward(torch.tensor(1.0))  
$X=\left\{ x_{ij} \right\}_{3 \times 4}$  
$A=\left\{ a_{ij} \right\}_{3 \times 3}$  
$Y=\left\{ X^T\cdot A\cdot X \right\}_{4 \times 4}$  
$Y=sum(diag(Y))=tr(X^T\cdot A\cdot X)$  
$\left. \frac{dY}{dX} \right. =\left\{(A+A^T)X \right\}_{3 \times 4}$  
$\left. \frac{dY}{dA} \right. =(X\cdot X^T)_{3 \times 3}$  

In [11]:
X=torch.arange(12, dtype=torch.float64).reshape(3,4)
A=torch.randn(3,3, dtype=torch.float64,requires_grad=True)
X.requires_grad_(True)
Y=torch.mm(torch.mm(X.T,A),X)
gradient=torch.eye(4,4, dtype=torch.float64)
Y.backward(gradient=gradient)
DY_DX=torch.mm((A+A.T),X)
DY_DA=torch.mm(X,X.T)
X,A,Y,gradient
print(X.grad,'\n',DY_DX)
print(A.grad,'\n',DY_DA)

tensor([[-11.5539, -12.2023, -12.8507, -13.4991],
        [ 12.0690,  14.9666,  17.8642,  20.7619],
        [-18.2809, -21.9504, -25.6199, -29.2894]], dtype=torch.float64) 
 tensor([[-11.5539, -12.2023, -12.8507, -13.4991],
        [ 12.0690,  14.9666,  17.8642,  20.7619],
        [-18.2809, -21.9504, -25.6199, -29.2894]], dtype=torch.float64,
       grad_fn=<MmBackward0>)
tensor([[ 14.,  38.,  62.],
        [ 38., 126., 214.],
        [ 62., 214., 366.]], dtype=torch.float64) 
 tensor([[ 14.,  38.,  62.],
        [ 38., 126., 214.],
        [ 62., 214., 366.]], dtype=torch.float64, grad_fn=<MmBackward0>)


分离计算  
$y=f(x)$  
$z=h(x,y)=h(x,f(x))=g(x)$  
计算$\frac{dz}{dx}= \frac{\partial h}{\partial x}+ \frac{\partial h}{\partial f} \times \frac{\partial f}{\partial x}$  
$\left. \frac{dz}{dx} \right|_{y}=\left. \frac{\partial h}{\partial x}+
 \frac{\partial h}{\partial f} \right|_{y} \times \left. \frac{\partial f}{\partial x} \right|_{y}$  
希望将y视为一个常数， 并且只考虑到x在y被计算后发挥的作用,即令  
$\left. \frac{\partial h}{\partial f} \right|_{y}=0$  从而计算$\left. \frac{dz}{dx} \right|_{y}=\left. \frac{\partial h}{\partial x} \right.$  

In [12]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u,u.grad


(tensor([True, True, True, True]), None)

还可以单独计算$\frac{dy}{dx}$

In [13]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使是非常复杂的函数代码，也可以求梯度  
并且分段函数(一阶导数不连续)也可以求，因为梯度是针对某一个特定点求出来的

In [14]:
def f(a):
    """ f(a)=d*a,d取决于a """
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()
a.grad == d / a


tensor(True)

<pre>特性               backward()                  torch.autograd.grad()
返回值​             无返回值（None）                返回梯度元组
梯度存储​           累积到叶子节点的.grad属性       不修改.grad属性，直接返回梯度值
计算图处理​          默认释放计算图                 默认释放计算图
适用输出​           标量输出                        标量或非标量输出
主要用途​           神经网络训练                    需要直接使用梯度值的场景

In [15]:
import torch

# 准备相同的数据和计算
x1 = torch.tensor(2.0, requires_grad=True)
x2 = torch.tensor(2.0, requires_grad=True)  # 用于torch.autograd.grad

# 相同的计算图构建
y1 = x1 ** 3 + 2 * x1  # 使用backward
y2 = x2 ** 3 + 2 * x2  # 使用torch.autograd.grad

print("初始状态:")
print(f"x1.grad: {x1.grad}, x2.grad: {x2.grad}")  # 都是None

# 使用backward()计算梯度
y1.backward()
print("\n--- 使用 y1.backward() 后 ---")
print(f"x1.grad: {x1.grad}")  # 梯度存储在x1.grad中

# 使用torch.autograd.grad()计算梯度
grad_x2 = torch.autograd.grad(outputs=y2, inputs=x2)
print("\n--- 使用 torch.autograd.grad(y2, x2) 后 ---")
print(f"grad_x2: {grad_x2}")  # 返回梯度元组
print(f"x2.grad: {x2.grad}")  # x2的grad属性仍为None

初始状态:
x1.grad: None, x2.grad: None

--- 使用 y1.backward() 后 ---
x1.grad: 14.0

--- 使用 torch.autograd.grad(y2, x2) 后 ---
grad_x2: (tensor(14.),)
x2.grad: None


In [16]:
# 继续上面的例子，再次计算梯度
y1_new = x1 ** 2  # 新的计算
y2_new = x2 ** 2  # 新的计算

# backward()会累积梯度
y1_new.backward()
print("\n--- 再次 y1_new.backward() 后 ---")
print(f"x1.grad: {x1.grad}")  # 14 + 4 = 18

# torch.autograd.grad()不会累积梯度
grad_x2_new = torch.autograd.grad(outputs=y2_new, inputs=x2)
print("\n--- 再次 torch.autograd.grad(y2_new, x2) 后 ---")
print(f"grad_x2_new: {grad_x2_new}")  # 只返回当前梯度4
print(f"x2.grad: {x2.grad}")  # 仍为None


--- 再次 y1_new.backward() 后 ---
x1.grad: 18.0

--- 再次 torch.autograd.grad(y2_new, x2) 后 ---
grad_x2_new: (tensor(4.),)
x2.grad: None


两种方法默认都会释放计算图，都需要设置retain_graph=True来保留：

In [ ]:
x = torch.tensor(3.0, requires_grad=True)
y = x ** 2

# 以下会报错，因为计算图在第一次反向传播后被释放
try:
    y.backward()
    y.backward()  # 第二次会报错
except RuntimeError as e:
    print(f"错误: {e}")

# 使用retain_graph=True避免错误
x_safe = torch.tensor(3.0, requires_grad=True)
y_safe = x_safe ** 2
y_safe.backward(retain_graph=True)  # 保留计算图
print(f"\n第一次反向传播后 x_safe.grad: {x_safe.grad}")  # 第一次的梯度
y_safe.backward()  # 可以再次反向传播
print(f"\n安全执行后 x_safe.grad: {x_safe.grad}")  # 累积的梯度

错误: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

第一次反向传播后 x_safe.grad: 6.0

安全执行后 x_safe.grad: 12.0


当输出不是标量时，两种方法的处理方式不同：

In [22]:
x = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y = x ** 2  # y是向量 [1, 4, 9]

# backward()需要指定gradient参数
gradient = torch.tensor([1.0, 1.0, 1.0])
y.backward(gradient)  # 相当于对y.sum()求导
print(f"backward()结果: {x.grad}")  # [2, 4, 6]

# torch.autograd.grad()更直接
x2 = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
y2 = x2 ** 2
grad_result = torch.autograd.grad(outputs=y2, inputs=x2, grad_outputs=torch.tensor([1.0, 1.0, 1.0]))
# 输出一个元组
print(f"torch.autograd.grad()结果: {grad_result[0]}")

backward()结果: tensor([2., 4., 6.])
torch.autograd.grad()结果: tensor([2., 4., 6.])


计算二阶导数

In [25]:
x = torch.tensor(2.0, requires_grad=True)
y = x ** 3

# 计算一阶导数
y.backward(create_graph=True)  # 保留计算图用于高阶导数
print(f"一阶导数: {x.grad.item()}")  # 12.0

# 在计算二阶导数前，需要从一阶导数继续反向传播
x.grad.zero_()  # 清空当前梯度
x.grad.backward()  # 对一阶导数进行反向传播
print(f"二阶导数: {x.grad.item()}")  # 12.0

# 使用后及时清理，避免内存泄漏
x.grad = None  # 打破引用循环[1](@ref)

一阶导数: 12.0
二阶导数: 0.0


<pre>使用 torch.autograd.grad()方法
这种方法更为直接和功能纯粹，它直接返回梯度值，而不修改原始张量的 .grad属性。
基本步骤：
    计算一阶导：调用 torch.autograd.grad计算一阶导数，并必须设置 create_graph=True。
    计算二阶导：以第一步返回的一阶导数张量作为新的 outputs，再次调用 torch.autograd.grad计算二阶导。
关键特性：
    无梯度累积：torch.autograd.grad()仅返回梯度值，不会自动累积到任何张量的 .grad属性中，因此无需手动清零梯度，避免了意外的梯度叠加
    灵活性高：由于直接返回梯度张量，这种方法在需要精确控制梯度计算流程的复杂场景（如元学习、自定义二阶优化器）中非常有用。

对于高阶导数计算，推荐使用torch.autograd.grad()而非backward(create_graph=True)，因为前者能有效避免内存泄漏问题，并且使用更加简洁安全


In [24]:
x = torch.tensor(2.0, requires_grad=True)
y = x ** 3  # 定义函数 y = x^3

# 第一步：计算一阶导数，并创建其计算图
first_deriv = torch.autograd.grad(y, x, create_graph=True)[0]
print(f"一阶导数: {first_deriv}")  # 输出：tensor(12., grad_fn=<CopyBackwards>)

# 第二步：直接从一阶导数计算二阶导数
second_deriv = torch.autograd.grad(first_deriv, x)[0]
print(f"二阶导数: {second_deriv}")  # 输出：tensor(12.)

一阶导数: 12.0
二阶导数: 12.0
